In [33]:
import pymysql

# your username and password go here!
creds = {'user': 'kevin', 
         'pswd': 'osullivan'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [2]:
createRevision = '''CREATE TABLE IF NOT EXISTS revision (
                page_id INT(8) UNSIGNED NOT NULL,
                week INT(8) UNSIGNED NOT NULL,
                num_revisions INT(8) UNSIGNED NOT NULL,
                num_reversions INT(8) UNSIGNED NOT NULL,
                num_mentions_of_vandalism INT(8) UNSIGNED NOT NULL,
                edit_protected VARCHAR(255) NOT NULL,
                move_protected VARCHAR(255) NOT NULL,
                PRIMARY KEY (page_id, week)
                ) DEFAULT CHARACTER SET utf8'''

In [3]:
cur.execute(createRevision)

0

In [2]:
cols_for_db = ['page_id', 'week', 'num_revisions', 'num_mentions_of_vandalism', 'edit_protected', 'move_protected']

In [3]:
import pandas as pd
epa_articles = pd.read_csv("../WikiSQL/RevisionsOverTime_ep.csv")
nep_articles = pd.concat([pd.read_csv("../WikiSQL/RevisionsOverTime_nep_Sample%d.csv" % i) for i in range(1, 9)])

In [4]:
epa_pro = pd.read_csv('../WikiSQL/Proections_ep.csv')
nep_pro = pd.concat([pd.read_csv("../WikiSQL/Proections_nep_Sample%d.csv" % i) for i in range(1, 9)])

In [5]:
import sys
sys.path.insert(2, '../WikiSQL/')
from CheckProtections import *
epa_pro_edit, epa_pro_move = GetProtectionPeriods(epa_pro)
nep_pro_edit, nep_pro_move = GetProtectionPeriods(nep_pro)

In [6]:
epa_articles[['num_reversions', 'num_mentions_of_vandalism']] = \
epa_articles[['num_reversions', 'num_mentions_of_vandalism']].astype(int)

nep_articles[['num_reversions', 'num_mentions_of_vandalism']] = \
nep_articles[['num_reversions', 'num_mentions_of_vandalism']].astype(int)

In [7]:
epa_articles['week'] = [WeekNum2DateStr(wk) for wk in epa_articles['week']]
epa_articles['weekdate'] = [date(int(str(wk)[0:4]), int(str(wk)[4:6]), int(str(wk)[6:8])) \
                            for wk in epa_articles['week']]
epa_articles['edit_protected'] = [IsWeekProtected(wk, epa_pro_edit[pid], True) if pid in epa_pro_edit.keys() \
                                  else IsWeekProtected(wk, [], True) for wk, pid in \
                                  zip(epa_articles['weekdate'], epa_articles['page_id'])]
epa_articles['move_protected'] = [IsWeekProtected(wk, epa_pro_move[pid], True) if pid in epa_pro_move.keys() \
                                  else IsWeekProtected(wk, [], True) for wk, pid in \
                                  zip(epa_articles['weekdate'], epa_articles['page_id'])]

nep_articles['week'] = [WeekNum2DateStr(wk) for wk in nep_articles['week']]
nep_articles['weekdate'] = [date(int(str(wk)[0:4]), int(str(wk)[4:6]), int(str(wk)[6:8])) \
                              for wk in nep_articles['week']]
nep_articles['edit_protected'] = [IsWeekProtected(wk, nep_pro_edit[pid], False) if pid in nep_pro_edit.keys() \
                        else IsWeekProtected(wk, [], False) for wk, pid in \
                        zip(nep_articles['weekdate'], nep_articles['page_id'])]
nep_articles['move_protected'] = [IsWeekProtected(wk, nep_pro_move[pid], False) if pid in nep_pro_move.keys() \
                        else IsWeekProtected(wk, [], False) for wk, pid in \
                        zip(nep_articles['weekdate'], nep_articles['page_id'])]

In [8]:
redundant = pd.merge(epa_articles, nep_articles, how='inner', left_on="page_id", right_on="page_id")
redundantIDs = redundant['page_id'].tolist()
nep_articles_filtered = nep_articles.loc[~nep_articles['page_id'].isin(redundantIDs)].copy()

In [9]:
master_articles = pd.concat([epa_articles, nep_articles_filtered])

In [10]:
master_articles = master_articles.where((pd.notnull(master_articles)), None) #converts NaNs to None type, required for mySQL

In [11]:
master_articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8637371 entries, 0 to 663415
Data columns (total 8 columns):
page_id                      int64
week                         int64
num_revisions                int64
num_reversions               int64
num_mentions_of_vandalism    int64
weekdate                     object
edit_protected               object
move_protected               object
dtypes: int64(5), object(3)
memory usage: 593.1+ MB


In [12]:
ma_for_db = master_articles[cols_for_db].to_dict(orient='records') #provides each row as a dict;
#keys are colnames, values are column values, the order of keys is retained from row to row

In [34]:
cur.execute("SET autocommit=0") #makes batch inserts SO MUCH EASIER

0

In [35]:
#generates the template query for insertions into db
insertQ = "INSERT IGNORE INTO revisions({}) VALUES ({})"
cols = ",".join(list(ma_for_db[0].keys()))
val_format = ",".join(['%s'] * len(list(ma_for_db[0].keys())))
formatQ = insertQ.format(cols, val_format)
formatQ

'INSERT IGNORE INTO revisions(num_mentions_of_vandalism,move_protected,num_revisions,edit_protected,page_id,week) VALUES (%s,%s,%s,%s,%s,%s)'

In [36]:
from time import time
t0 = time()
for record in ma_for_db[4000000::]:
    values = list(record.values())
    cur.execute(formatQ,values)

print(time() - t0)
connection.commit()

31144.86182808876


/Users/dekedor/anaconda/lib/python3.4/site-packages/pymysql/cursors.py:134: Warning: Field 'num_reversions' doesn't have a default value
  result = self._query(query)


In [44]:
interestingCols = ['page_id','week','num_revisions','edit_protected']
interestingQ = "SELECT {} FROM revisions"
colNameString = ",".join(interestingCols)
preppedIQ = interestingQ.format(colNameString)

cur.execute(preppedIQ)
rows = cur.fetchall() #returns tuple of tuples, preserving column order

easyPeasy = pd.DataFrame(list(rows), columns=interestingCols)
easyPeasy

/Users/dekedor/anaconda/lib/python3.4/site-packages/pymysql/cursors.py:134: Warning: Field 'num_reversions' doesn't have a default value
  result = self._query(query)


,page_id,week,num_revisions,edit_protected


In [33]:
cur.close()
connection.close()

In [23]:
len(ma_for_db)

8637371

In [15]:
ma_for_db[551435]

{'edit_protected': 'unknown',
 'move_protected': 'not protected',
 'num_mentions_of_vandalism': 0,
 'num_revisions': 8,
 'page_id': 45246,
 'week': 20140202}